# File Permissions

What are file permissions? File permissions determine which users can operate on files on a system. These operations are broken down into three categories:

- Read: Viewing the contents of a file
- Write: Changing the contents of a file
- Execute: Running contents of a file as code
- Permissions function differently when applied to directories. 

1. Read allows users to view the contents of a directory, 
2. write allows users to add and delete files, and 
3. execute allows users to cd into that directory.

NOTE: There are three more permissions on top of the ones explained above: setuid, setgid, and sticky bit. 

What are file permissions?

It is important to properly assign permissions to files and directories created on a shared system so that files are not accidentally (or maliciously) tampered with, destroyed, or executed. The next section shows you how to view the permissions of files in an example directory.

Displaying files and their permissions

```bash
cd /srv
```

The command ls -l can be used to look at the permissions associated with each file in the directory. The output for this can be cryptic at first, so this step will walk you through what each character means. Take a look at the contents of the current directory. Use the long listing option (-l) to display information about permissions for each item.

```bash
ls -l

drw-r-----. 2 root root  94 Jun  2 21:08 proprietary
-rwxr-x---. 1 root root  66 Jun  2 21:08 status.sh
-rwxr-x---. 1 root root 117 Jun  2 21:08 tasks.txt
```

The first column of the output above is a string conveying the access mode. The access mode -rwxr-x---, seen here for the status.sh file, conveys the following information:

<p style="text-align: center">
  <img  src="Media/RWX.png" width="600" alt="read-write-execute">
</p>

The middle columns of the ls output give information about the user ownership and group ownership of the files, their size, the time last modified, and finally the file/directory name itself. The next steps will walk you through the importance of the file ownership fields.




Interacting with a file from different user accounts
Recall from the previous step that status.sh has the following permissions:

```bash
-rwxr-x---
```

Therefore, root has permission to execute this script.

```bash
./status.sh
```

status.sh successfully executed by root
Focus now on line 2, the line for status.sh. The first three letters of the access mode show that the owner of the file has full permissions, rwx, so the owner can read, write, and execute this file. Users in the group that owns this file have r-x, so they can read and execute but cannot write to this file. Finally, all other users have no permissions, ---, so they are unable to read, write, or execute this file.

To see this in action, switch users to the guest account using the su command:
```bash
su - guest
```

Confirm that this command has succeeded by looking at your bash prompt, the user should be guest.

```bash
[guest@rhel ~]$
```

Navigate back to the directory containing the status.sh script so that you can run it as this new user.

```bash
cd /srv
```

Try executing the status script as guest. Since the guest account is not the user owner of the file and is not part of any owner groups, you are unable to execute this script from this user account.

```bash
./status.sh
bash: ./status.sh: Permission denied
```

Now that you know how files behave differently when operated on by different user accounts, the next steps will show you how to customize permissions to control this behavior.

The chmod command is how you modify the permissions associated with files and directories. The chmod command has two different ways to modify permissions: symbolic and absolute. Symbolic permissions have the same format as the access mode output of ls -l. In this mode, you use the characters r, w, and x to set the read, write, and execute permissions. The other mode, absolute, instead uses a series of three numbers to correspond to the permissions for the owner, group, and others. These numbers are identical in meaning to the symbolic mode breakdown, they are just more compact and therefore quicker to type. Here is an example of how the two modes relate from Boolean World:

<p style="text-align: center">
  <img  src="Media/Chmod.png" width="600" alt="chmod">
</p>

NOTE: This image refers to absolute mode as numeric mode. These two terms are interchangeable and you will commonly see both.

The crucial takeaway is that each character in a symbolic permission can be converted to a binary number, 1 if the user has that permission and 0 if the user does not. These three binary numbers become a single octal digit, and three such octal digits compose an absolute representation of the file's permissions.

## Changing permissions (symbolic)
In this step, you will be modifying the permissions on the status.sh script using symbolic syntax with the chmod command. In the previous step, you were unable to execute status.sh as rhel. To change that, use the chmod command. The plus (+) operator will add any permissions that you specify, but it will not change any permissions that you do not explicitly set in the command. So in this case, it will give others read and execute access, but will not change the value of write access for the group. Return to the root terminal to execute this command.

```bash
chmod o+rx status.sh
```

Verify that this command succeeded using ls -l, specifying that you want to only look at status.sh.

```bash
ls -l status.sh
-rwxr-xr-x. 1 root root  66 Jun  2 22:42 status.sh
```

NOTE: Just as o+rx added read and execute permissions for others, chmod accepts the - argument to take away permissions. For example, o-w would remove write privileges from others while leaving the read and execute values untouched. The = argument can be used to explicitly set all permissions symbolically.

Now the guest user will be able to read and execute status.sh. Switch back to the guest terminal:

```bash
su - guest
```

and re-run the status script:

```bash
cd /srv
./status.sh
status.sh successfully executed by guest
```

Since you added read and execute permissions, you can do more than just execute this file as guest. Use the cat command to verify that you can read this file as guest.

```bash
cat status.sh
#!/bin/bash
echo -n "status.sh successfully executed by "
whoami
```

Symbolic mode is the more expanded permissions format which can appeal to newer users. However, the absolute mode described in the next step can save on keystrokes if you take the time to familiarize yourself with it.

Now your next goal is to modify the permissions on status.sh to keep full access for the owner and group members, but prevent anyone else from writing to or executing the file. You can do this with three numbers if you use the absolute syntax.

The number you would use to do this with chmod is 774. The image below explains how the absolute permission of 774 relates to the access mode string that is output by ls -l.

<p style="text-align: center">
  <img  src="Media/744.png" width="600" alt="744">
</p>

## Changing permissions (absolute)
The owner user and owner group will have full permissions, but others will only be able to read the file. Return to the first terminal where you are logged in as root. Use chmod to modify the permissions on status.sh as discussed above:

Confirm that this change has succeeded by checking the access mode for status.sh

```bash
ls -l status.sh
-rwxrwxr--. 1 root root  66 Jun  2 22:42 status.sh
```

Now, from the guest terminal, verify that you do not have permission to execute the status script as guest:

```bash
./status.sh
bash: ./status.sh: Permission denied
```

However, if you try to read this file, you will see you do have read access as guest:

```bash
cat status.sh
#!/bin/bash
echo -n "status.sh successfully executed by "
whoami
```

NOTE: You can use vi or any other write command to verify that you do not have write access as guest if you wish to verify this as well.

Sometimes it is useful to change who owns a file so that permissions can align with the purpose of the file. The chown command can do this very easily. Just specify the new owner and the file you would like to change.

Changing file ownership
From the root terminal, confirm that root is the initial owner of the file tasks.txt:

```bash
ls -l tasks.txt
-rwxr-x---. 1 root root 117 Jun  2 23:01 tasks.txt
```

The third and fourth column of this output are the user and group that own the file. Right now, root is the owner in both cases. Switch to the guest terminal and attempt to read the file:

```bash
cat tasks.txt
cat: tasks.txt: Permission denied
```

This read operation is denied, as guest does not have read permission. Returning to the root terminal, change the user that owns tasks.txt to guest:

```bash
chown guest tasks.txt
```

Repeating the search shows that the owner (column 3) has been updated:

```bash
ls -l tasks.txt
-rwxr-x---. 1 guest root 117 Jun  2 23:01 tasks.txt
```

Now that guest owns the file instead of root, this user has permission to read the file without reconfiguring the access modes.

```bash
cat tasks.txt
```

The fourth column of the ls -l output is the group which owns the file. The next step will walk you through changing group ownership.

Changing group ownership can be much more efficient than manually assigning owners, when there are many users that should have the same access to a file.

Changing Group Ownership
Group ownership can be changed with the chgrp command by adding a colon before the name of the group. Return to the root terminal to execute this command:

```bash
chown :team tasks.txt
```

Verify the change using ls -l:

```bash
ls -l tasks.txt
-rwxr-x---. 1 rhel team 117 Jun  2 23:01 tasks.txt
```

This change is reflected in the fourth column of the ls -l output, which now says team instead of root.

NOTE: Some people prefer the chgrp command for changing groups as it has identical syntax to chown, you do not have to add a colon when using chgrp. Others prefer to use chown with the colon as above so that they do not have to remember two different commands.

The final step in this lab will walk you through using find to locate any files that may have their permissions configured incorrectly.

The find command is highly versatile, but in this case you will be using it to locate files that have permissions that may pose security risks.

## Auditing permissions
The -perm option causes the find command to look at the access modes for each file. In this case you are calling find without a path, which means it will start looking in the current directory. First, check for files that have the permission level 755.

```bash
find -perm 755
```

This returns only a hidden OS file:

```bash
.
```

NOTE: Recall from the previous step that 755 corresponds to read, write, and execute access for the owner, and read and execute access for everyone else.

Consider a situation where you have determined that any permissions above a certain level could pose a security issue. Adding a dash before the permission tells the find command to return anything with a permission greater than or equal to the one you are searching for.

```bash
find -perm -755
.
./proprietary/contract01.txt
./proprietary/contract02.txt
./proprietary/contract03.txt
./proprietary/contract04.txt
```

Notice that this returns many more files. You may not have seen these files from the output of ls -l, but find searches recursively so it can find files that otherwise may slip through the cracks. The contract files in the proprietary directory should not be readable by everyone. Furthermore, since these are text files, they should not be executable.

Running the chmod command with a level of 600 ensures that these files are readable and writable by the owner, but others cannot access them. Instead of running chmod individually on each file, you can use the find command to execute this permission change on any file that exceeds a certain permission level. The -type f option only targets files, so the directory permissions will remain unchanged. The {} \; at the end of the command is telling find to execute the chmod command separately on each file that matches the search criteria.

```bash
find -perm -755 -type f -exec chmod 600 {} \;
```

To verify that access has been removed, try to read one of the files as the guest account. Switch to the guest terminal and run this command:

```bash
cat proprietary/contract01.txt

cat: proprietary/contract01.txt: Permission denied
```

NOTE: You may be tempted to use chmod with the recursive option (-R) to make this change instead of using find. The chmod -R command modifies subdirectories as well as files, however. Since you are removing execute permissions, using chmod recursively would lead to a situation where you could not cd into any of the subdirectories that you just modified. This would also prevent you from running any recursive operations to fix this issue, and you would have to manually change the permissions on each subdirectory individually.

It is very important to double check your work when using the find to execute chmod, as running this command on the wrong directory could lock users out of all of their files.